In [1]:
import cv2 
import os
import numpy as np
import pandas as pd
from imutils import paths
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
    
from tensorflow.keras import layers, models
import csv

print(tf.test.is_built_with_cuda())
print(tf.config.list_physical_devices('GPU'))

2024-04-03 13:06:17.737615: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-03 13:06:17.850644: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-03 13:06:18.262746: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/rtx4090-2204/.local/lib/python3.10/site-packages/cv2/../../lib64:/usr/lib/x86_64-linux-gnu:/

True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-04-03 13:06:18.785306: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-03 13:06:18.816428: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-03 13:06:18.816500: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
from pathlib import Path

dataset_folder = 'dataset'

image_paths = sorted(Path(dataset_folder).glob('*.png'), key=lambda x: int(x.stem))

img_data = []

for image_path in image_paths:
    print(f"Processing image: {image_path}")
    try:
        image = cv2.imread(str(image_path))
        if image is not None:
            image = cv2.resize(image, (240, 240))
            img_data.append(image)
        else:
            print(f"Unable to read image: {image_path}")
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")

img_data = np.array(img_data)

print(f"Total images processed: {len(img_data)}")
print(f"Shape of img_data: {img_data.shape}")

img_data = img_data / 255.0


Processing image: dataset/0.png
Processing image: dataset/1.png
Processing image: dataset/2.png
Processing image: dataset/3.png
Processing image: dataset/4.png
Processing image: dataset/5.png
Processing image: dataset/6.png
Processing image: dataset/7.png
Processing image: dataset/8.png
Processing image: dataset/9.png
Processing image: dataset/10.png
Processing image: dataset/11.png
Processing image: dataset/12.png
Processing image: dataset/13.png
Processing image: dataset/14.png
Processing image: dataset/15.png
Processing image: dataset/16.png
Processing image: dataset/17.png
Processing image: dataset/18.png
Processing image: dataset/19.png
Processing image: dataset/20.png
Processing image: dataset/21.png
Processing image: dataset/22.png
Processing image: dataset/23.png
Processing image: dataset/24.png
Processing image: dataset/25.png
Processing image: dataset/26.png
Processing image: dataset/27.png
Processing image: dataset/28.png
Processing image: dataset/29.png
Processing image: da

In [3]:
Label_data = []

csv_paths = "data.csv"

with open(csv_paths, 'r') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        Label_data.append(row['JoystickValue'])

Label_data = np.array(Label_data)

print(Label_data.shape)

(7144,)


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(img_data, Label_data, test_size=0.2, random_state=5)

# Split the remaining data into validation and test dataset
X_val, X_test, y_val, y_test = train_test_split(X_rem, y_rem, test_size=0.5, random_state=5)

# Convert the datasets into TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

# Print shapes
for images, labels in train_dataset.take(1):
  print('X_train shape:', images.shape)
  print('y_train shape:', labels.shape)

for images, labels in val_dataset.take(1):
  print('X_val shape:', images.shape)
  print('y_val shape:', labels.shape)

for images, labels in test_dataset.take(1):
  print('X_test shape:', images.shape)
  print('y_test shape:', labels.shape)


2024-04-03 13:07:11.718240: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-03 13:07:11.719606: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-03 13:07:11.719695: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-03 13:07:11.719733: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at

In [5]:
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')
print(y_train.shape)
print(y_test.shape)
print(y_train.dtype)
print(y_test.dtype)

(5715,)
(1429,)
float32
float32


In [6]:
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.layers import Conv2D, Flatten, Dense

def tf_model(input_shape):
    model = tf.keras.Sequential([
        Conv2D(24, (5, 5), activation='relu', input_shape=input_shape),
        Conv2D(36, (5, 5), activation='relu'),
        Conv2D(48, (5, 5), activation='relu'),
        Flatten(),
        Dense(100, activation='relu'),
        Dense(50, activation='relu'),
        Dense(1) 
    ])
    return model

input_shape = (240,240,3)
model = tf_model(input_shape)

model.compile(optimizer=optimizers.Adam(learning_rate=0.0003), loss='mse')


2024-03-16 12:13:40.395216: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-16 12:13:40.395951: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-16 12:13:40.396045: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-16 12:13:40.396084: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 236, 236, 24)      1824      
                                                                 
 conv2d_1 (Conv2D)           (None, 232, 232, 36)      21636     
                                                                 
 conv2d_2 (Conv2D)           (None, 228, 228, 48)      43248     
                                                                 
 flatten (Flatten)           (None, 2495232)           0         
                                                                 
 dense (Dense)               (None, 100)               249523300 
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 1)                 5

In [8]:
from keras.callbacks import Callback
import warnings

class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor='val_loss', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current_val = logs.get(self.monitor)
        if current_val is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)
            return

        if current_val < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True

early_stopping = EarlyStoppingByLossVal(monitor='val_loss', value=0.00001, verbose=1)

In [9]:
def data_generator(X, y, batch_size):
    num_samples = len(X)
    while True:
        indices = np.random.choice(num_samples, size=batch_size, replace=False)
        batch_X = X[indices]
        batch_y = y[indices]
        yield batch_X, batch_y

batch_size = 72
epochs = 300
validation_split = 0.2

train_generator = data_generator(X_train, y_train, batch_size)

history = model.fit_generator(
    generator=train_generator,
    epochs=epochs,
    steps_per_epoch=len(X_train) // batch_size,  # Number of batches per epoch
    callbacks=[early_stopping],
    verbose=1
)


Epoch 1/300


/tmp/ipykernel_139396/3325520755.py:15: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(
2024-03-16 12:13:41.023044: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8907
2024-03-16 12:13:42.040666: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-03-16 12:13:42.051504: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0xdcd5370 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-16 12:13:42.051516: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-03-16 12:13:42.054113: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `M

79/79 [==============================] - 8s 74ms/step - loss: 16.1562
Epoch 2/300
 2/79 [..............................] - ETA: 5s - loss: 0.1508

/tmp/ipykernel_139396/1713847602.py:14: RuntimeWarning: Early stopping requires val_loss available!
  warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)


79/79 [==============================] - 6s 74ms/step - loss: 0.1546
Epoch 3/300
79/79 [==============================] - 6s 74ms/step - loss: 0.1361
Epoch 4/300
79/79 [==============================] - 6s 74ms/step - loss: 0.1193
Epoch 5/300
79/79 [==============================] - 6s 74ms/step - loss: 0.0969
Epoch 6/300
79/79 [==============================] - 6s 74ms/step - loss: 0.0828
Epoch 7/300
79/79 [==============================] - 6s 74ms/step - loss: 0.0712
Epoch 8/300
79/79 [==============================] - 6s 74ms/step - loss: 0.0573
Epoch 9/300
79/79 [==============================] - 6s 75ms/step - loss: 0.0500
Epoch 10/300
79/79 [==============================] - 6s 75ms/step - loss: 0.0391
Epoch 11/300
79/79 [==============================] - 6s 75ms/step - loss: 0.0303
Epoch 12/300
79/79 [==============================] - 6s 75ms/step - loss: 0.0289
Epoch 13/300
79/79 [==============================] - 6s 75ms/step - loss: 0.0258
Epoch 14/300
79/79 [================

In [ ]:
tf.keras.backend.clear_session()